In [1]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()

openai.api_type = "azure"
openai.api_version = "2022-12-01"

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT
COMPLETIONS_MODEL = os.getenv('DEPLOYMENT_NAME')
print(COMPLETIONS_MODEL)
print(RESOURCE_ENDPOINT)

text-davinci-003
https://claroshop-open-ai-sc.openai.azure.com/


In [9]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

#### Completion API

In [4]:
prompt = """Answer the question as truthfully as possible, and if you're unsure of the answer, say "Sorry, I don't know".

Q: Who won the 2020 Summer Olympics men's high jump?
A:"""
openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=1000,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    engine=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

"Sorry, I don't know."

In [5]:
prompt = """Decide whether the following customer feedback is positive or negative.

Q: I was disappointed with the quality of the product. It was very cheaply made and did not meet my expectations at all.
Q: I was happy with this product, it is well made and great quality for the price.
"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    engine=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'Negative'

In [ ]:
prompt = """List all PII data from following statement:
John Doe is a 35-year old man and he lives at 21 Main Street, New York, NY. He is a software engineer and he works at Google. He has a wife named Jane Doe and they have two children
"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    engine=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

#### Code Generation

In [ ]:
response = openai.Completion.create(
  engine=COMPLETIONS_MODEL,
  prompt="### Postgres SQL tables, with their properties:\n#\n# Employee(id, name, department_id)\n# Department(id, name, address)\n# Salary_Payments(id, employee_id, amount, date)\n#\n### A query to list the names of the departments which employed more than 10 employees in the last 3 months\n\ query: SELECT",
  temperature=0,
  max_tokens=150,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])

print(response.choices[0].text)

In [ ]:
response = openai.Completion.create(
  engine=COMPLETIONS_MODEL,
  prompt="Can you explain what does this code do?\n#\n# ###\n\
   Code:\n\
   SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10\n#\n#\
   Answer:\n# ",
   max_tokens=250,
  stop=["#",";"])

print(response.choices[0].text)

#### Token Usage

In [10]:
prompt = "Azure OpenAI service is General Available now!"
tokens = tokenizer(prompt)
print('Total number of tokens:', len(tokens['input_ids']))
print('Tokens : ', [tokenizer.decode(t) for t in tokens['input_ids']])

Total number of tokens: 10
Tokens :  ['Az', 'ure', ' Open', 'AI', ' service', ' is', ' General', ' Available', ' now', '!']


In [11]:
response = openai.Completion.create(
    engine=COMPLETIONS_MODEL,
    prompt=prompt,
    max_tokens=60,
    n=2,
    best_of=2,
)

In [12]:
print('='*30, 'ANSWER #1', '='*30)
print(response['choices'][0]['text'])
print('='*30, 'ANSWER #2', '='*30)
print(response['choices'][1]['text'])

============================== ANSWER #1 ==============================


Yes, Azure OpenAI is now generally available. This cloud service allows developers and data scientists to access the latest deep learning technology from OpenAI. This includes access to OpenAI's algorithm libraries, compute resources, model training, and more. The benefits of using OpenAI as a cloud service
============================== ANSWER #2 ==============================


Yes, the Azure OpenAI service is now generally available. Microsoft announced the news at a recent event, celebrating the launch of Azure’s OpenAI service. The service allows developers to easily take advantage of an open AI platform that can scale up to their needs. With the service,


In [13]:
response['usage']

<OpenAIObject at 0x263ec020c70> JSON: {
  "completion_tokens": 120,
  "prompt_tokens": 10,
  "total_tokens": 130
}

#### OpenAI Parameters

##### 
Temperature - Defaults to 1

What sampling temperature to use. Higher values means the model will take more risks. Try 0.9 for more creative applications, and 0 (argmax sampling) for ones with a well-defined answer.

We generally recommend altering this or top_p but not both.

In [15]:
def call_openai(num_times, prompt, temperature):
    for i in range(num_times):
        
        response = openai.Completion.create(
            engine=COMPLETIONS_MODEL,
            prompt=prompt,
            max_tokens=60,
            temperature = temperature
        )
        print(response['choices'][0]['text'])

In [16]:
call_openai(10, 'The best pet is a ', temperature = 0)


dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, 

In [17]:
call_openai(10, 'The best pet is a ', temperature = 1)


dog. Dogs are loyal and affectionate and make great companions for people of all ages and lifestyles. They are easy to train and can be tailored to fit any lifestyle with the proper obedience training. Dogs offer unconditional love and protection as well as companionship and can provide hours of entertainment.

dog.

dog. Dogs are loyal, loving companions that provide both emotional and physical health benefits to their owners. They can help reduce stress, keep you active and be great sources of joy and company.


The best pet is a matter of personal preference. Many people enjoy having cats, dogs, birds, fish, reptiles, and small mammals as long as they can provide them with a loving home and proper care. Some people prefer more independent pets while others prefer pets that require more attention and companions

dog. Dogs are loyal companions that can provide a lifetime of unconditional love and companionship. They can be trained to perform a variety of tasks, providing mental stimu

##### 
top_p - Defaults to 1

An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.

We generally recommend altering this or temperature but not both.

In [18]:
def call_openai(num_times, prompt, top_p):
    for i in range(num_times):
        
        response = openai.Completion.create(
            engine=os.getenv('DEPLOYMENT_NAME'),
            prompt=prompt,
            max_tokens=60,
            top_p = top_p
        )
        print(response['choices'][0]['text'])

In [19]:
call_openai(10, 'The best pet is a ', top_p = 1)


dog.

Dogs are loyal, loving, and can provide a wonderful sense of companionship. They are also great for providing exercise and are often easy to train.

dog. Dogs provide excellent companionship, offer unconditional love, can help build confidence and provide protection. They are also low maintenance, relatively inexpensive, and require only a small amount of time and effort to care for.


dog.

dog. Dogs are loyal, loving, and playful companions, and they make great additions to any family. Plus, they’re easy to train, so you’ll have a pup that listens, knows the house rules and is eager to please.

dog. Dogs are loyal, loving companions and make great family pets. They provide unconditional love and support and can provide hours of companionship and fun. Dogs also help teach responsibility and provide exercise and outdoor time for their owners.


goldfish. Goldfish are cheerful, low-maintenance, and they come in all sorts of colors. They can live in small tanks and they don't requ

In [20]:
call_openai(10, 'The best pet is a ', top_p = 0.1)


dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, 

##### 
n - Defaults to 1

How many completions to generate for each prompt.

Note: Because this parameter generates many completions, it can quickly consume your token quota. Use carefully and ensure that you have reasonable settings for max_tokens and stop.

In [22]:
response = openai.Completion.create(
            engine=COMPLETIONS_MODEL,
            prompt='The best pet is a ',
            max_tokens=60,
            n=2
        )

for c in response['choices']:
    print(c['text'])



dog. Dogs are loyal, loving, and always up for a walk or playtime. They can provide lots of emotional support and are a great companion to have around.

dog. Dogs provide unconditional love and loyalty, making them excellent companions and loyal friends. They are also relatively low maintenance, compared to some other pets, and can bring joy and happiness to any home.


##### 
logprobs - Defaults to null

Include the log probabilities on the logprobs most likely tokens, as well the chosen tokens. For example, if logprobs is 5, the API will return a list of the 5 most likely tokens. The API will always return the logprob of the sampled token, so there may be up to logprobs+1 elements in the response.

The maximum value for logprobs is 5. If you need more than this, please contact us through our Help center and describe your use case

In [23]:
response = openai.Completion.create(
            engine=COMPLETIONS_MODEL,
            prompt='The best pet is a ',
            max_tokens=60,
            logprobs = 2,
        )

print(response['choices'][0]['logprobs'])
print(response['choices'][0]['text'])

{
  "text_offset": [
    18,
    19,
    20,
    23,
    24,
    29,
    34,
    40,
    51,
    52,
    56,
    62,
    63,
    67,
    71,
    79,
    85,
    88,
    92,
    96,
    110,
    114,
    120,
    127
  ],
  "token_logprobs": [
    -0.0076072733,
    -1.0273676,
    -0.32800227,
    -0.10885598,
    -0.24196677,
    -2.2397401,
    -0.9036527,
    -0.18346992,
    -0.95600116,
    -0.80736697,
    -0.55934405,
    -0.75776064,
    -0.16180483,
    -0.3973285,
    -0.5767172,
    -2.1588624,
    -0.00032121275,
    -1.3243638,
    -0.078839876,
    -0.5485573,
    -3.4078758,
    -0.37301013,
    -0.039342094,
    -0.009277409
  ],
  "tokens": [
    "\n",
    "\n",
    "dog",
    ".",
    " Dogs",
    " make",
    " great",
    " companions",
    ",",
    " are",
    " loyal",
    ",",
    " and",
    " can",
    " provide",
    " hours",
    " of",
    " fun",
    " and",
    " entertainment",
    " for",
    " their",
    " owners",
    "."
  ],
  "top_logprobs": [
    

#####
presence_penalty
Defaults to 0

Number between -2.0 and 2.0. Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.

##### 
frequency_penalty
Defaults to 0

Number between -2.0 and 2.0. Positive values penalize new tokens based on their existing frequency in the text so far, decreasing the model's likelihood to repeat the same line verbatim.

##### 
best_of
Defaults to 1

Generates best_of completions server-side and returns the "best" (the one with the highest log probability per token). Results cannot be streamed.

When used with n, best_of controls the number of candidate completions and n specifies how many to return – best_of must be greater than n.

Note: Because this parameter generates many completions, it can quickly consume your token quota. Use carefully and ensure that you have reasonable settings for max_tokens and stop.

##### 
logit_bias
Defaults to null

Modify the likelihood of specified tokens appearing in the completion.

Accepts a json object that maps tokens (specified by their token ID in the GPT tokenizer) to an associated bias value from -100 to 100. You can use this tokenizer tool (which works for both GPT-2 and GPT-3) to convert text to token IDs. Mathematically, the bias is added to the logits generated by the model prior to sampling. The exact effect will vary per model, but values between -1 and 1 should decrease or increase likelihood of selection; values like -100 or 100 should result in a ban or exclusive selection of the relevant token.

As an example, you can pass {"50256": -100} to prevent the <|endoftext|> token from being generated.